# Notebook Summary

## Introduction

Use probabilites produced in notebook 03 to create useful data products for SOC assignement.

         
## Output/Results

1. List of prioritised standards
   - `f'nsfg_data/soc_assignment_prioritisation_20230222.xlsx'`


2. Information for RMs to determine best match for top prioritised standards
   - `nsfg_data\export_results_prio_Construction and the built environment_v3.2.1__20230222_1019.xlsx`

## Recommendation for Viewing Code

- The headings in this notebook follow Markdown convention i.e # means H1 and ## is H2
- I recommend adding a Table of Contents add on to see the structure of this notebook in the best way
- Here is a great package that can add a ToC to jupyter notebooks: https://github.com/minrk/ipython_extensions


# Notebook Setup

## Constants

In [ ]:
run_version = 'v4.2.2' 
training_data_version = run_version.split('.')[0]
label = 'autoassign_in_top_3'
best_model = 'model_b'
prob_col = f'{best_model}_output_prob'

## Regular Imports

In [ ]:
### Imports 
import os
import sys
from pathlib import Path
import datetime

import numpy as np
import math
from scipy import stats

import pandas as pd
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## My Imports

In [ ]:
# Add Path of where you have imported my functions
current_path = os.getcwd()
functions_path = Path(current_path, '..', 'functions')
sys.path.append(str(functions_path))

In [ ]:
## Pandas functions
import laurie_pandas_functions as pd_funcs
from laurie_pandas_functions import display_full

## Matplotlib funcs
import laurie_plotting_functions as plot_funcs
from laurie_plotting_functions import get_ax, force_ax_grid

In [ ]:
## Useful when developing your functions
from importlib import reload  
reload(pd_funcs)
reload(plot_funcs)

In [ ]:
def run_ls_on_path(path):
    """
    Run ls on a path in jupyter and display to notebook
    Can't be imported as uses cell magic
    Args: path (pathlib.WindowsPath): path created by pathlib
    """
    userhome = os.path.expanduser("~")
    reformatted_path = ('\"' + str(path).replace('\\\\', '\"/\"') + '\"').replace('\"~\"','~').replace('~', userhome)
    print(f'$ ls {path}')
    !ls {reformatted_path}
    print('\n')

## Plotting Setup

In [ ]:
### Colours
blue =  '#79a5f7'
red  =  '#ff9696'
green=  '#9ebd9e'
sns_colours = sns.color_palette()

### Make the plots a nice size
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'large',
          'axes.labelsize': 'large',
          'axes.titlesize':'large',
          'xtick.labelsize':'large',
          'ytick.labelsize':'large',
          'figure.titlesize':'x-large'}
pylab.rcParams.update(params)

# Get Data
Read the csvs from the paths and show the top two rows

## Predictions

In [ ]:
path_prediction_export = Path('nsfg_data', f'df_output_modelling_{run_version}.csv')
df_predictions = pd.read_csv(path_prediction_export)

## Raw features

In [ ]:
path_features_import = Path('nsfg_data', f'df_train_data_nlp__{training_data_version}.csv')
df_features = pd.read_csv(path_features_import)

## Add URL

In [ ]:
api_url = 'https://www.instituteforapprenticeships.org/api/apprenticeshipstandards'

import requests

def query_url_to_df(url):
    response = requests.get(url)
    result = response.json()
    data = pd.DataFrame.from_records(result)
    return data

df_api_raw = query_url_to_df(api_url)

In [ ]:
df_api_snaked = pd_funcs.apply_snake_case_to_df(df_api_raw)

In [ ]:
api_to_standard_renamer = {
    'reference_number': 'standard_code',
    'versionNumber': 'version',
    'standard_page_url': 'standard_page_url',
}

df_api_renamed = (
    df_api_snaked
    .rename(columns=api_to_standard_renamer)
    [api_to_standard_renamer.values()]
    .assign(version=lambda df: df['version'].astype(float))
)

# Process Data

## Merge

In [ ]:
prediction_cols = list(set(df_predictions.columns) - set(df_features.columns))
merge_cols = ['standard_code', 'version', 'soc_2020_ext_code']

In [ ]:
df_display = (
    df_features
    .merge(
        df_predictions[merge_cols+prediction_cols],
        how='left',
        on=merge_cols,
        indicator='_merge_with_prediction',
    )
    .merge(
        df_api_renamed,
        how='left',
        on=['standard_code', 'version'],
        indicator='_merge_with_api',
    )

)

In [ ]:
pd_funcs.agg_df_by_cols(df_display, '_merge_with_prediction', display_df=True)
pd_funcs.agg_df_by_cols(df_display.drop_duplicates('standard_code'), '_merge_with_api', display_df=True)


In [ ]:
df_display = df_display.drop(['_merge_with_prediction', '_merge_with_api'], axis=1)

## Add some new columns

In [ ]:
df_display['soc_major_group_expected_range'] =  (df_display['soc_major_group_lower'].astype(str) + ' - ' + df_display['soc_major_group_upper'].astype(str)).str.replace('.0', '')

In [ ]:
df_display['ids_soc_2020_code'] = np.where(
    df_display['ids_soc_2020_code'].isna(),
    'N/A',
    df_display['ids_soc_2020_code'].astype(str).str[:4],
)

In [ ]:
yes_no_cols = [
    'soc_2020_matches_previous_assignment',
    'soc_in_suggested_major_group',
]

for c in yes_no_cols:
    df_display[c] = np.where(
        df_display[c] > 0,
        'Yes',
        np.where(
            df_display[c] == 0,
            'No',
            'N/A'
        )
    )

In [ ]:
df_display['match_rank_label'] = np.where(
    df_display['match_rank'].isna(),
    '',
    np.where(
        df_display['match_rank'] == 1,
        'Primary',
        'Alternative #' + (df_display['match_rank'].fillna(0) - 1).astype(int).astype(str),
    )
)

In [ ]:
dict_rayg_label = {
    'g': 'Strong Match',
    'y': 'Medium Match',
    'a': 'Weak Match',
    'r': 'Unlikely to be a match',
}

dict_rayg_colours = {
    'g': '#9ebd9e',
    'y': '#ffff96',
    'a': '#ffcb96',
    'r': '#ff9696',
}

case_when_prob_col_label = {
    dict_rayg_label['g']: df_display[prob_col] > 0.5,
    dict_rayg_label['y']: df_display[prob_col] > 0.25,
    dict_rayg_label['a']: df_display[prob_col] > 0.02,
    dict_rayg_label['r']: df_display[prob_col] >= -0.01,
}

df_display['prob_col_label'] = np.select(condlist=case_when_prob_col_label.values(), choicelist=case_when_prob_col_label.keys(), default='N/A')

In [ ]:
pd_funcs.agg_df_by_cols(df_display.loc[lambda df: df['rank_model_b'] <= 10], label, set_cols='prob_col_label', display_df=True, do_total=False, sort_by_cols=True)

## Level Range by SOC Major Group
- No-one will know what SOC major group is
- But everyone will know what Level is
- So easy to say is standard within expected levels.

In [ ]:
# Get Param's from Jody's file
df_level_params = pd.read_csv('data/soc_group_level_parameters.csv')

In [ ]:
## Adjust from level to soc mg range to soc mg to level range
min_soc_mg = df_level_params['soc_major_group_lower'].min()
max_soc_mg = df_level_params['soc_major_group_upper'].max()

soc_major_groups = []
min_levels = []
max_levels = []

for s in range(min_soc_mg, max_soc_mg+1):
    
    df_range = (
        df_level_params
        .loc[lambda df: df['soc_major_group_lower'] <= s]
        .loc[lambda df: df['soc_major_group_upper'] >= s]
    )
    
    soc_major_groups.append(s)
    min_levels.append(df_range['level'].min())
    max_levels.append(df_range['level'].max())

df_soc_major_group_params = pd.DataFrame(
    {
        'soc_2020_major_group': soc_major_groups,
        'level_lower': min_levels,
        'level_upper': max_levels,
    }

)

df_soc_major_group_params

In [ ]:
### Merge to display
df_display = (
    df_display
    .merge(df_soc_major_group_params, on='soc_2020_major_group', suffixes=('_', ''))
)


df_display['within_level_expected_range'] = np.where(
    (df_display['level'] >= df_display['level_lower']) & (df_display['level'] <= df_display['level_upper']),
    'Yes',
    'No',
)

df_display['within_level_expected_range_display'] =  df_display['within_level_expected_range'] + (' (L' + df_display['level_lower'].astype(str) + ' - ' + df_display['level_upper'].astype(str)+')').str.replace('.0', '')


level_range_cols = ['level_lower', 'level_upper']
for c in level_range_cols:
    old_col = f'{c}_'
    if old_col in df_display:
        df_display = df_display.drop(old_col, axis=1)


In [ ]:
df_display

# Prioritasation

## Prioritasation Metrics

In [ ]:
df_display['status']

In [ ]:
df_prio = (
    df_display
    .assign(
        prob_of_top_gatsby_match=lambda df: np.where(
            df['autoassign_is_top'],
            df[prob_col],
            None,
        ),
        prob_of_gatsby_match=lambda df: np.where(
            df['autoassign_is_ranked'],
            df[prob_col],
            None,
        ),
        prob_of_gatsby_match_low_pct=lambda df: np.where(
            df['autoassign_is_ranked'],
            (df[prob_col] < 0.02).astype(int),
            None,
        ),
        top_gatsby_match_soc_2020_ext_code=lambda df: np.where(
            df['autoassign_is_top'],
            df['soc_2020_ext_code'],
            None,
        ),
        top_gatsby_match_soc_2020_ext_title=lambda df: np.where(
            df['autoassign_is_top'],
            df['soc_2020_ext_title'].astype(str),
            '',
        ),
    )
    .groupby(['route', 'standard_code', 'standard_title'])
    .agg(**{
            'max_prob': (prob_col, max),
            'min_prob_of_gatsby_match': ('prob_of_gatsby_match', min),
            'prob_of_top_gatsby_match': ('prob_of_top_gatsby_match', max),
            'mean_prob_of_gatsby_match': ('prob_of_gatsby_match', 'mean'),
            'n_low_prob_gatsby_matches': ('prob_of_gatsby_match_low_pct', sum),
            'top_gatsby_match_soc_2020_ext_title': ('top_gatsby_match_soc_2020_ext_title', max),
            'n_matches': ('autoassign_is_ranked', 'sum'),
            'status': ('status', 'max'),
        }
    )  
    .assign(
        delta_in_max_prob=lambda df: df['max_prob'] - df['prob_of_top_gatsby_match'],
        delta_mean_prob_of_gatsby_match=lambda df: df['max_prob'] - df['mean_prob_of_gatsby_match'],
    )
    .assign(
        prio_low = lambda df: df['n_low_prob_gatsby_matches']
    )
    .assign(
        prio_low_norm = lambda df: df['prio_low'] / df['prio_low'].max()
    )
    .assign(
        prio_reweighted = lambda df: (df['delta_in_max_prob'] + df['prio_low_norm']) / 2 + (1 - df['mean_prob_of_gatsby_match']) * 0.001
    )
    .assign(
        prio_ranked = lambda df: df['prio_reweighted'].rank(method='first', ascending=False)
    )
    .reset_index()
)

In [ ]:
df_prio['prio_rank_display'] = np.where(
    df_prio['prio_ranked'].isna(),
    'Un-ranked',
    (df_prio['prio_ranked'].astype(str) + ' (of ' + str(df_prio.shape[0]) + ')').str.replace(r'\.0', ''),
)

## View Prio

In [ ]:
display_full(
    df_prio
    .sort_values(['prio_ranked'])
    .head(5000)
    .set_index(['prio_ranked', 'standard_code'])
    [['prio_rank_display', 'route', 'standard_title', 'prio_reweighted', 'delta_in_max_prob', 'prio_low_norm', 'n_low_prob_gatsby_matches', 'prob_of_top_gatsby_match', 'mean_prob_of_gatsby_match']]
    , force=True
)

## Investigate Prioritisation By Route

In [ ]:
df_prio = (
    df_prio
    .assign(
        standards_in_top_100 = lambda df: df['prio_ranked'] <= 100,
        standards_in_top_250 = lambda df: df['prio_ranked'] <= 250,
    )
)

df_prio_by_route = (
    df_prio
    .groupby('route')
    .agg(**{
        'standards': ('standard_code', 'count'),
        'standards_in_top_100': ('standards_in_top_100', sum),
        'standards_in_top_250': ('standards_in_top_250', sum),
    }
    )
    .assign(
        pct_standards_in_top_100 = lambda df: df['standards_in_top_100'] * 100 / df['standards'],
        pct_standards_in_top_250 = lambda df: df['standards_in_top_250'] * 100 / df['standards'],
    )
)

In [ ]:
cols = ['standards', 'standards_in_top_100', 'pct_standards_in_top_100']

st_cols = ['standards']
top_cols = [c for c in cols if c.startswith('standards')]
rel_cols = [c for c in cols if c.startswith('pct_standards_in')]

(
    df_prio_by_route
    [cols]
    .style
    .format('{:,.1f}%', subset=rel_cols)
    .bar(width=70, align='mid', subset=st_cols, color=blue)
    .bar(width=70, align='mid', subset=top_cols, color=green)
    .bar(width=70, align='mid', subset=rel_cols, color=red)
    .set_table_styles(pd_funcs.get_lauries_table_styles())
)

In [ ]:
cols = ['standards', 'standards_in_top_250', 'pct_standards_in_top_250']

st_cols = ['standards']
standards_in_top_cols = [c for c in cols if c.startswith('standards_in')]
rel_cols = [c for c in cols if c.startswith('pct_standards_in')]

(
    df_prio_by_route
    [cols]
    .style
    .format('{:,.1f}%', subset=rel_cols)
    .bar(width=70, align='mid', subset=st_cols, color=blue)
    .bar(width=70, align='mid', subset=standards_in_top_cols, color=green)
    .bar(width=70, align='mid', subset=rel_cols, color=red)
    .set_table_styles(pd_funcs.get_lauries_table_styles())
)

## Add to df_display

In [ ]:
df_display = (
    df_display
    .merge(
        df_prio[['standard_code', 'prio_rank_display']],
        on='standard_code',
        how='left',
        suffixes=('__', '')
    )
)

if 'prio_rank_display__' in df_display.columns:
    df_display = df_display.drop('prio_rank_display__', axis=1)

In [ ]:
df_prio

# Export Prio

In [ ]:
dict_prio_display_rename = {
    'route': 'Route', 
    'standard_code': 'ST Code', 
    'standard_title': 'Standard Title', 
    'prio_ranked': 'Priority Rank',
    'delta_in_max_prob': 'Difference in probability between Gatsby and AI\'s Top Match',
    'n_low_prob_gatsby_matches': 'Number of Unlikely Gatsby Matches',
    'status': 'Standard Status',
}

df_prio_displayed = (
    df_prio
    .loc[lambda df: df['prio_ranked'] <= 250]
    .sort_values('prio_ranked')
    .assign(prio_ranked = lambda df: df['prio_ranked'].astype(int))
    .assign(delta_in_max_prob = lambda df: (df['delta_in_max_prob']*100.0).round(1).astype(str)+'%')
    [dict_prio_display_rename.keys()]
    .rename(columns=dict_prio_display_rename)
    .set_index(dict_prio_display_rename['prio_ranked'])
)

In [ ]:
today = datetime.datetime.today().strftime('%Y%m%d')
df_prio_displayed.to_excel(f'nsfg_data/soc_assignment_prioritisation_{today}.xlsx')

# Export to Excel

## Constants

In [ ]:
display_full(df_display.head(3).T)

In [ ]:
standard_cols = [
    'standard_code',
    'standard_title',
    'route',
    'prio_rank_display',
    'level',
    'standard_overview',
    'option_titles'    
]

soc_cols = [
    'soc_2020_ext_code',
    'soc_2020_ext_title',
    'match_rank_label',
    'prob_col_label',
    'soc_job_matches_standard_title',
    'soc_2020_ext_description',
    'standard_typical_job_title',
    'soc_job_matches_typical_job',
    'within_level_expected_range_display',    
]

cols_dictionary = {
    'standard_code': 'Standard Code',
    'standard_title': 'Standard Title',
    'soc_2020_ext_code': 'SOC2020 Ext Code',
    'soc_2020_ext_title': 'SOC2020 Ext Title',
    'match_rank_label': 'Current Assignment',
    'prio_rank_display': 'QA Priority Rank',
    prob_col: 'IfATE Model Probability',
    'prob_col_label': 'IfATE Model Assessment',
    'soc_2020_matches_previous_assignment': 'SOC2020 consistent with previous assignment',
    'ids_soc_2020_code': 'SOC 2020 Assigned on IDS',
    'ids_soc_2020_rationale': 'IDS SOC 2020 Rationale',
    'score_soc_job_match_standard_title': 'Similarity - SOC Job & Standard Title',
    'soc_job_matches_standard_title': 'Soc Job Role Matched to Standard Title',
    'score_soc_job_match_typical_job': 'Similarity - Standard Typical Job Role & SOC Job',
    'standard_typical_job_title': 'Best match between Standard Typical Job Titles and SOC Job Roles',
    'soc_job_matches_typical_job': '',
    'score_overview': 'Similarity Description',
    'standard_overview': 'Standard Overview',
    'soc_2020_ext_description': 'SOC2020 Ext Description',
    'soc_in_suggested_major_group': 'SOC in suggested major group',
    'level': 'Level',
    'soc_major_group_expected_range': 'Expected SOC Major Group Range',
    'soc_2020_major_group': 'SOC 2020 Major Group',
    'route': 'Route',
    'standard_page_url': 'IfATE Webpage',
    'within_level_expected_range_display': 'Standard meets Expected Level Range for this SOC Code',
    'option_titles': 'Option Titles',
}

cols_meaning_dictionary = {
    'standard_code': 'The ST code of the Apprenticeship Standard',
    'standard_title': 'The Apprenticeship Standard Title',
    'soc_2020_ext_code': 'The 6 Digit SOC2020 Sub-unit Group Code',
    'soc_2020_ext_title': 'SOC2020 Sub-unit Group Name',
    'match_rank_label': 'The Current Assignment in IDS, which is initially taken from a data-set provided by Gatsby, a independant body.',
    'prio_rank_display': 'What priority for QA has been assigned to this Standard',
    'prob_col_label': (
        'For every standard IfATE runs an AI model that independently assesses how strong the match between the SOC 2020 SUG code and Apprenticeship Standard. '
        'This is then transformed to a RAYG assessment of the strength of the match, where Green means likely to be a match, and Red means unlikely to be a match. '
        'This model can accelerate the mapping process, but bare in mind that this model is sometimes wrong, so use good judgement.'   
    ),
    'soc_job_matches_standard_title': 'Soc Job Role Matched to Standard Title',
    'standard_typical_job_title': 'Best match between Standard Typical Job Titles and SOC Job Roles',
    'soc_job_matches_typical_job': '',
    'standard_overview': 'Standard Overview',
    'soc_2020_ext_description': 'SOC2020 Ext Description',
    'level': 'Level',
    'route': 'Route',
    'standard_page_url': 'IfATE Webpage',
    'within_level_expected_range_display': (
        'The SOC 2020 Major Group (1st digit of the code) indicates how senior this role is, '
        'and we can infer the range of Standard levels that are usually related to this level of seniority. '
        'This column shows if the standard meets the expected level range, '
        'and the full expected level range is shown in brackets.'
    ),
    'option_titles': 'Titles of Options (if Standard is Core and Options)',
}

In [ ]:
## Intro text above column description on explanation text
creation_time = str(datetime.datetime.today())[:16]

intro_text = [
    ['This dataset provides the information to accelerate the process of identifying the 6-digit SOC 2020 Sub-Unit Group (SUG) Codes from ONS that best represent our Occupational Standards.'],
    [f'Creation Date:  {creation_time}'],
    [''],
    ['Context:'],
    ['-> IfATE has commited to keeping an up-to-date registry of which SOC 2020 SUG codes best represent the Occupational Standards, and this will be published via the Occupational Maps'],
    ['-> This will enable the whole education data ecosystem to link to our products and vice-versa'],
    ['-> Find more about SOC 2020 Extension Codes at', 'https://www.ons.gov.uk/methodology/classificationsandstandards/standardoccupationalclassificationsoc/standardoccupationalclassificationsocextensionproject'],
    ['-> Find more about Occupational Standards at', 'https://www.instituteforapprenticeships.org/occupational-standards/'],
    [''],
    ['Data Explanation:'],
    ['-> At the top we show information about the standard and a link to the Standard page, which contains more information about the Standard.'],
    ['-> Then, we show useful links for information that help the process'],
    ['-> Below, we show SOC SUG codes that are current stored in IDS as matches to the Standard. AI model assesses the strength of each map in the form of a RAYG rating is shown. Then there is other information that relates to the quality of match.'],
    ['-> Finally, we show the 20 SOC SUG codes that are most likely to match to the Standard according to the AI. This is to enable us to quickly check that there are no better matches.'],
    ['-> Most of the time 1-3 SOC 2020 SUG Codes should be sufficient to cover the Occupation, although occasionaly more is required, especially for core and options standards.'],
    [''],
    ['Column descriptions:'],
]

## Functions

In [ ]:
import datetime

In [ ]:
title_cols = ['standard_title', 'soc_2020_ext_title', 'soc_job_matches_standard_title', 'standard_typical_job_title', 'soc_job_matches_typical_job', 'route', 'prob_col_label', 'within_level_expected_range_display']
title_cols = [cols_dictionary[c] for c in title_cols]
        
medium_text_cols = ['standard_overview', 'ids_soc_2020_rationale']
medium_text_cols = [cols_dictionary[c] for c in medium_text_cols]

long_text_cols = ['soc_2020_ext_description']
long_text_cols = [cols_dictionary[c] for c in long_text_cols]
long_text_cols.append('Delivery Comments')

In [ ]:
def write_df_to_excel(data, writer, sheet_name, startrow, index=True):
    
    header_format = writer.book.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'border': 1}
    )
    
    if data.shape[0] == 0:
        print('No Data Available')
        worksheet = writer.sheets[sheet_name]
        worksheet.write(startrow , 0, 'No Data Available')

    
    else:
        
        data_styled = data.style
        
        data.to_excel(writer, encoding='utf-8', sheet_name=sheet_name, startrow=startrow+1, header=False, index=index)

        worksheet = writer.sheets[sheet_name]
        if index:
            header_names = list(data.index.names) + list(data.columns.values)
        else:
            header_names = list(data.columns.values)
            
        for col_num, value in enumerate(header_names):
            worksheet.write(startrow, col_num, value, header_format) 
    


In [ ]:
def create_intro(writer, list_text, dict_col_descriptions, sheet_name_intro):
    """
    Function to write an intro/explanation page
    
    Args:
    - writer (pd.ExcelWriter): The excel writer object
    - list_text (list of list of strings): List of of list of text to put in cells. Outer list represents rows, inner list represents columns in that row.
    - dict_explanations (dict): Dict representating col variable name and explanation.
    - sheet_name_intro (str): Name of sheet to put explanation in.
    """
    
    cols_used = standard_cols + soc_cols
    col_names = []
    col_descriptions = []
    for c in cols_used:
        if c != '':
            col_names.append(cols_dictionary[c])
            col_descriptions.append(dict_col_descriptions[c])
        
    df_descriptions = pd.DataFrame({'Column Name': col_names, 'Explanation': col_descriptions})
    
    start_row_descriptions = len(list_text)
    end_row_descriptions = len(list_text) + len(df_descriptions) + 1
    write_df_to_excel(df_descriptions, writer, sheet_name=sheet_name_intro, startrow=start_row_descriptions, index=False)

    workbook = writer.book
    worksheet = writer.sheets[sheet_name_intro]    
    
    for y, line in enumerate(list_text):
        for x, cell in enumerate(line):
            worksheet.write(y, x, cell)

    ## Set up format
    wrap_format = workbook.add_format({'text_wrap': True, 'align': 'left'})
    worksheet.set_column(0, 0, width=60)
    worksheet.set_column(1, 1, width=100, cell_format=wrap_format)


In [ ]:
def create_contents(writer, st_codes, df_display, sheet_name_contents='Contents'):
    """
    Function to write an contents page
    
    Args:
    - writer (pd.ExcelWriter): The excel writer object
    - st_codes (list of  strings): List of ST codes of form 'ST1234' which are the prioritised list.
    - df_display: The main df that we can use.
    - sheet_name_contents (str): Name of sheet to put contents in.
    """

    index = ['standard_code']
    index_standard = [cols_dictionary[c] for c in standard_cols if c in index]

    delivery_comments = 'Delivery Comments'
    
    cols_for_contents = [
        cols_dictionary[c] 
        for c in standard_cols 
        if c not in index
        if c not in ['standard_overview', 'standard_page_url', 'level'] # Longer name cols
    ] 
    cols_for_contents += [delivery_comments]

    # Add empty column for coments
    data = df_display.assign(**{delivery_comments: ''})
    
    df_standards = (
        data
        .rename(columns=cols_dictionary)
        .set_index(index_standard) 
        [cols_for_contents]
        .drop_duplicates()
        .loc[st_codes]
    )  
    
                 
    write_df_to_excel(df_standards, writer, sheet_name=sheet_name_contents, startrow=2, index=True)

    workbook = writer.book
    wrap_format = workbook.add_format({'text_wrap': True, 'align': 'left'})

    worksheet = writer.sheets[sheet_name_contents]
    
    max_n_columns = df_standards.shape[1]
    
    add_table_title_row(workbook, worksheet, 0, max_n_columns, 'List of standards in this file:')


    for i, col in enumerate(index_standard + cols_for_contents):
                        
        if (col in long_text_cols) or (col == cols_dictionary['option_titles']):
            width = 55
        elif (col in title_cols):
            width = 25
        else:
            width = 12
                
        if i == 0:
            cell_format = None
        else:
            cell_format = wrap_format
                
        worksheet.set_column(i, i, width=width, cell_format=cell_format)


In [ ]:
def add_table_title_row(workbook, worksheet, i, max_j, text_0, text_1=''):
    
    bold_grey_format = workbook.add_format({'bold': True, 'bg_color': '#D9D9D6', 'top': 1})
    
    for j in range(max_j + 1):
        
        if j == 0:
            text = text_0
        elif j == 1:
            text = text_1
        else:
            text = ''
            
        worksheet.write(i, j, text, bold_grey_format)



In [ ]:
def add_links(worksheet, dict_links, st_code, ifate_url, start_row):    
    
    for i, (link_name, link_url) in enumerate(dict_links.items()):
        i_ = start_row + i
        worksheet.write(i_, 0, link_name)
        
        link_url = link_url.format(st_code=st_code, ifate_url=ifate_url)
        worksheet.write(i_, 2, link_url)
    
    

In [ ]:
def output_to_excel(df_display, dict_links, st_codes, path):

    index = ['standard_code', 'soc_2020_ext_code']
    index_soc = [cols_dictionary[c] for c in soc_cols if c in index]
    index_standard = [cols_dictionary[c] for c in standard_cols if c in index]
    n_auto_suggestions = 20
    
    cols_to_show_soc = [cols_dictionary[c] for c in soc_cols if c not in index]
    cols_to_show_standard = [cols_dictionary[c] for c in standard_cols if c not in index]
    
    with pd.ExcelWriter(path) as writer:
        
        create_intro(writer, list_text=intro_text, dict_col_descriptions=cols_meaning_dictionary, sheet_name_intro='Introduction')
        
        create_contents(writer, st_codes, df_display, sheet_name_contents='Contents')
        
        for i, st_code in enumerate(st_codes):
        
            print(f'- {st_code}')
    
            data = (
                df_display
                .assign(standard_title_=lambda df: df['standard_title'])
                .loc[lambda df: df['standard_code'] == st_code]
                .rename(columns=cols_dictionary)
                .assign(prob_col=lambda df: df[cols_dictionary[prob_col]])
            )
            
            this_cols_to_show_standard = list(cols_to_show_standard)
            if not data['is_core_and_options'].max():
                this_cols_to_show_standard.remove(cols_dictionary['option_titles'])
            
            for c in data.columns:
                if (c.startswith('Similarity')) or (c == cols_dictionary[prob_col]):
                    data[c] = np.where(
                        data[c].isna(),
                        'N/A',
                        (data[c] * 100).round(1).astype(str) + '%',
                    )
            
            data = data.fillna('N/A')
            
            data_standard = (
                data
                .head(1)
                .set_index(index_standard)
                [this_cols_to_show_standard]
            )
            
    
            data_suggestions = (
                data
                .sort_values('prob_col', ascending=False)
                .set_index(index_soc)
                [cols_to_show_soc]
                .head(n_auto_suggestions)
            )
    
            data_gatsby_suggestions = (
                data
                .loc[lambda df: df[cols_dictionary['match_rank_label']].astype(str) != '']
                .sort_values('match_rank', ascending=True)
                .set_index(index_soc)
                [cols_to_show_soc]
            )
            
            n_data_standard = data_standard.shape[0]
            n_gatsby_suggestions = data_gatsby_suggestions.shape[0]
            n_links = len(dict_links)
    
    
            start_row_standard = 2
            write_df_to_excel(data_standard, writer, sheet_name=st_code, startrow=start_row_standard)
            
            start_row_links = start_row_standard + n_data_standard + 5
            
            start_row_gatbsy = start_row_links + n_links + 4
            write_df_to_excel(data_gatsby_suggestions, writer, sheet_name=st_code, startrow=start_row_gatbsy)

            start_row_auto = start_row_gatbsy + n_gatsby_suggestions + 5
            write_df_to_excel(data_suggestions, writer, sheet_name=st_code, startrow=start_row_auto)
            
            workbook  = writer.book
            wrap_format = workbook.add_format({'text_wrap': True, 'align': 'left'})
            title_format = workbook.add_format({'bold': True, 'border': True, 'valign': 'Top', 'text_wrap': True})
            
            worksheet = writer.sheets[st_code]
            max_n_columns = max(data_standard.shape[1], data_suggestions.shape[1], data_gatsby_suggestions.shape[1])
            add_table_title_row(workbook, worksheet, 0, max_n_columns, 'Table 1:', 'Standard Info:')
            add_table_title_row(workbook, worksheet, start_row_links-2, max_n_columns, 'Table 2:', 'Useful Links')
            add_table_title_row(workbook, worksheet, start_row_gatbsy-2, max_n_columns, 'Table 3:', 'Current SOC Assignments')
            add_table_title_row(workbook, worksheet, start_row_auto-2, max_n_columns, 'Table 4:', f'Top {n_auto_suggestions} AI Suggested Assignments')
            
            
            ifate_url = data[cols_dictionary['standard_page_url']].max()
            add_links(worksheet, dict_links, st_code=st_code, ifate_url=ifate_url, start_row=start_row_links)

            for i, col_soc in enumerate(index_soc + cols_to_show_soc):
                
                
                if i <= len(this_cols_to_show_standard):
                    col_standard = (index_standard + this_cols_to_show_standard)[i]
                else:
                    col_standard = None
                    
                if (col_standard in long_text_cols) or (col_soc in long_text_cols):
                    width = 60
                elif (col_standard in medium_text_cols) or (col_soc in medium_text_cols):
                    width = 20
                elif (col_standard in title_cols) or (col_soc in title_cols):
                    width = 15
                else:
                    width = 10
                
                if i == 0:
                    cell_format = None
                else:
                    cell_format = wrap_format
                
                if col_standard == cols_dictionary['option_titles']:
                    option_titles = data_standard[col_standard].max().replace('; ', '\n')
                    n_options = option_titles.count('\n') + 1
                    
                    option_column_name = col_standard
                    if n_options > 2:
                        option_column_name = f'{option_column_name} (Adjust cell height to see all)'
                        
                    worksheet.merge_range(start_row_standard, i, start_row_standard, i+2, option_column_name, title_format)
                    worksheet.merge_range(start_row_standard+1, i, start_row_standard+1, i+2, option_titles, wrap_format)
                    
                    #if data['is_core_and_options'].max():
                    #    worksheet.set_row(start_row_standard+1, max(2, n_options) * 15.0001)
                    
                if col_soc == cols_dictionary['standard_typical_job_title']:
                    if n_gatsby_suggestions > 0:
                        worksheet.merge_range(start_row_gatbsy, i, start_row_gatbsy, i+1, cols_dictionary['standard_typical_job_title'], title_format)
                    worksheet.merge_range(start_row_auto, i, start_row_auto, i+1, cols_dictionary['standard_typical_job_title'], title_format)

                worksheet.set_column(i, i, width=width, cell_format=cell_format)
                    
                if col_soc == cols_dictionary['prob_col_label']:
                    for k, colour in dict_rayg_colours.items():
                        colour_format = workbook.add_format({'bg_color': colour, 'border': True})
                        worksheet.conditional_format(
                            start_row_gatbsy, i, (start_row_auto + 21), i, 
                            {'type':     'cell',
                             'criteria': '=',
                             'value':  f'"{dict_rayg_label[k]}"',
                             'format': colour_format,
                            }
                        )                
                                
    print(f'Written to {path}')

In [ ]:
def create_assignment_table(df_display, st_codes, path, sheet_name='SOC Assignments'):
    
    index = ['standard_code']
    
    data_spine = (
        df_display
        .set_index('standard_code')
        .loc[st_codes]
        [['standard_title', 'route']]
        .assign(**{'Approved by RH': False})
        .drop_duplicates()
        .reset_index()
    )
    
    data_gatsby_suggestions = (
        df_display
        .loc[lambda df: df['standard_code'].isin(st_codes)]    
        .loc[lambda df: ~df['match_rank'].isna()]
        .sort_values('match_rank', ascending=True)
    )

    data_gatsby_suggestions_pivot = pd.pivot_table(
        data_gatsby_suggestions,
        index = index,
        columns='match_rank_label',
        values='soc_2020_ext_code',
        aggfunc=max,
        fill_value='',
    )
    
    # re-order cols
    max_rank = int(data_gatsby_suggestions['match_rank'].max())
    cols = ['Primary'] + [f'Alternative #{i}' for i in range(1, max_rank)]
    data_gatsby_suggestions_pivot = data_gatsby_suggestions_pivot[cols].reset_index()
    
    index = [cols_dictionary[c] for c in index]
    data_gatsby_suggestions_pivot = (
        data_spine
        .merge(
            data_gatsby_suggestions_pivot,
            how='left',
        )
        .rename(columns=cols_dictionary)
        .set_index(index)
    )
    
    with pd.ExcelWriter(path) as writer:                
        write_df_to_excel(data_gatsby_suggestions_pivot, writer, sheet_name=sheet_name, startrow=0)
        
        workbook  = writer.book
        
        wrap_format = workbook.add_format({'text_wrap': True, 'align': 'left'})
        
        worksheet = writer.sheets[sheet_name]
        worksheet.set_column(0, 20, width=10, cell_format=wrap_format)
        worksheet.set_column(1, 2, width=17, cell_format=wrap_format)
            
    print(f'Written to {path}')

In [ ]:
dict_links = {
    'IfATE Standard Page:': '{ifate_url}',
    'IDS for Data Entry:': 'https://ids/admin/summaries/summarysubmission/?q={st_code}',
    'Process Guidance:': 'https://educationgovuk.sharepoint.com/:w:/s/DDST/ESyFp4lTgydBlm3Af1M5Fu0BdeM99OhJKwffJ61QUZoRaQ?e=klKTmN',
}

In [ ]:
def create_results_table(df_display, st_codes, label):
    
    timestamp = datetime.datetime.today().strftime('%Y%m%d_%H%M')
    
    results_file_name = f'export_results_{label}_{run_version}__{timestamp}.xlsx'
    
    path_gatbsy_export_excel = Path('nsfg_data', results_file_name)
    output_to_excel(df_display, dict_links, st_codes, path_gatbsy_export_excel)    

## Test

In [ ]:
st_codes = ['ST0752', 'ST0003', 'ST1358', 'ST0585', 'ST0456']
create_results_table(df_display, st_codes, 'test')

## Top 20 Prio'd

In [ ]:
list_prio_st_codes = list(
    df_prio
    .sort_values('prio_ranked')
    .reset_index()
    .head(20)
    ['standard_code']
)  

In [ ]:
create_results_table(df_display, list_prio_st_codes, 'prio')

In [ ]:
list_low_prio_st_codes = list(
    df_prio
    .sort_values('prio_ranked', ascending=False)
    .reset_index()
    .head(20)
    ['standard_code']
)  
create_results_table(df_display, list_low_prio_st_codes, 'low_prio')

## Prio By Route

In [ ]:
routes = df_prio['route'].unique()

for this_route in routes:
    print(f'**** Doing {this_route} ****')
    list_prio_route_st_codes = (
        df_prio
        .reset_index()
        .loc[lambda df: df['route'] == this_route]
        .loc[lambda df: df['prio_ranked'] <= 250]
        .sort_values('prio_ranked', ascending=True)
        .reset_index()
        ['standard_code']
    )  
    create_results_table(df_display, list_prio_route_st_codes, f'prio_{this_route}')

In [ ]:
this_route = 'Digital'

list_prio_route_st_codes = (
    df_prio
    .reset_index()
    .loc[lambda df: df['route'] == this_route]
    .sort_values('prio_ranked', ascending=False)
    .head(10)
    .reset_index()
    ['standard_code']
)  

create_results_table(df_display, list_prio_route_st_codes, f'low_prio_{this_route}')